<img style="max-width:20em; height:auto;" src="../graphics/A-Little-Book-on-Adversarial-AI-Cover.png"/>

Author: Nik Alleyne   
Author Blog: https://www.securitynik.com   
Author GitHub: github.com/securitynik   

Author Other Books: [   

            "https://www.amazon.ca/Learning-Practicing-Leveraging-Practical-Detection/dp/1731254458/",   
            
            "https://www.amazon.ca/Learning-Practicing-Mastering-Network-Forensics/dp/1775383024/"   
        ]   


This notebook ***(knockoff-nets_v2_rate_limited_nftables_simplified.ipynb)*** is part of the series of notebooks From ***A Little Book on Adversarial AI***  A free ebook released by Nik Alleyne

### SIMPLIFIED VIEW OF THE RATE FILTERING WITHOUT THE ADDITIONAL COMMENTS   
- Feel free to copy and paste if you do not wish to type    
- If you do type, pay attention to the escape sequence "\" when using bash 

**Flush the ruleset**    
sudo nft flush ruleset    

**Verify no ruleset exists**   
sudo nft --numeric list ruleset    

**Create a table**   
sudo nft add table ip INFERENCE_EP { comment \" ML_INFERENCE_ENDPOINT PROTECTION \" \; }    

**Create the rules container**    
sudo nft add chain ip INFERENCE_EP input { type filter hook input priority 0 \; policy accept \; comment \"ML_INFERENCE_ENDPOINT RULES CONTAINER \" \; }    

**Create a named set to capture the suspicious IPs**    
sudo nft add set ip INFERENCE_EP suspicious_api_access { type ipv4_addr \; flags dynamic, timeout \; timeout 5m \; comment \" SUSPICIOUS IPS \" \; }     

**Create a rule to allow connections under the threshold**    
sudo nft add rule ip INFERENCE_EP input ip protocol tcp tcp dport 5000 ct state new, untracked limit rate over 50/minute add @suspicious_api_access { ip saddr } accept comment \" ML_INFERENCE_ENDPOINT incoming connections \"    

**Create a rule to drop traffic going across the threshold**    
sudo nft add rule ip INFERENCE_EP input ip saddr @suspicious_api_access tcp dport 5000 log prefix \" SUSPICIOUS CONNECTIONS DROPPED \" counter drop    


You can also validate that information is in your logs  
First by using **dmesg**: $ **sudo dmesg --ctime**   
then by $ **cat /var/log/syslog | grep SUSPICIOUS**   

### Optional to help with testing your firewalls:   
Testing our rule
Open or split your current window into 4 different screens.


let's start with installing hping3 
$ sudo apt-install hping3


In a separate window run tcpdump
$sudo tcpdump -nni any port 5000


In a separate window run a necat listener
$ncat --verbose --listen --keep-open 5000


# In another window make a connection to this netcat session to validate you can access the current listening session
$ ncat --verbose 127.0.0.1 5000


Assuming everything works as expected, you should have been able to make a successful connection from the client to the server. At the same time, you should see a successful 3-way handshake completed, which means your connection is now established.

Press CTRL+C to break out of the ncat client window


Now that we know that works, let's leverage HPing3 to generate some traffic

$ sudo hping3 --count 20 --baseport 9876 --destport 5000 --keep  --syn  --interface eth0 127.0.0.1 --faster


Now let's run HPing3 again this time increase the count parameter to a value of 100 or greater.
$ sudo hping3 --count 100 --baseport 9876 --destport 5000 --keep  --syn  --interface eth0 127.0.0.1 --faster


Now if we try to use ncat to connect to the listening netcat session, your connection has more than likely failed, assuming that we had our rules in place.  

$ sudo dmesg --ctime


$ If we instead wanted to follow the logs via syslog, we could do. 
$ tail --follow /var/log/syslog | grep kernel

---------------------


# Temporary disable the rules
nft> add table inet API_SECURITY { flags dormant; }


# Re-enable the rules
nft> add table inet API_SECURITY


------------

With this in place, we have significantly impede the threat actor's ability of being successful with many blackbox attacks against our API endpoint.